In [ ]:
import torch
import json
import pickle
import numpy as np
import pandas as pd

from torch_geometric.data import Data

In [ ]:
# Representação do grafo - Matriz de adjacência
# Por exemplo

# 0 é o vértice da imagem toda
# 1 ao 4 tenho os vértices referentes aos patches
# 5 é o vértice da pergunta toda
# a partir de 5 é o vértice de cada palavra

"""
#     visao      #        nlp        #
[0, 1, 2, ..., 49, 50, 51, ..., 60]
[1, ...                               ]
[2, ...                               ]
[..., ...                             ]
[49, ...                             ]
[50, ...                             ]
[..., ...                             ]
[60, ...                             ]

"""

# Atenção ao seguinte:
# - Self loop, ou seja, aresta do vértice pra ele mesmo (diagonal = 1)
# - Todas as palavras vão estar conectadas com todos os pedaços da imagem e imagem inteira
# - Conexões entre as palavras estão no arquivo

In [ ]:
def loadData(name_arq):

    data = {}
    with open(name_arq, 'rb') as fr:
        try:
            while True:
                data.update(pickle.load(fr))
        except EOFError:
            pass

    return data

def get_connections_patches():

    source = []
    target = []

    matrix_patch = np.array(list(range(1,5))).reshape((2,2))

    for p in range(1,5):

        pos = np.argwhere(matrix_patch == p).tolist()[0]
        row = pos[0]
        col = pos[1]

        if (row-1) >= 0:
            neighbour_up = matrix_patch[row-1, col]
        else:
            neighbour_up = -100
        if (row+1) < 2:
            neighbour_down = matrix_patch[row+1, col]
        else:
            neighbour_down = -100
        if (col-1) >= 0:
            neighbour_left = matrix_patch[row, col-1]
        else:
            neighbour_left = -100
        if (col+1) < 2:
            neighbour_right = matrix_patch[row, col+1]
        else:
            neighbour_right = -100

        neighbours = sorted([n for n in [neighbour_up, neighbour_down, neighbour_left, neighbour_right] if n != -100])

        source = source + [p]*len(neighbours)
        target = target + neighbours

    return source, target

def get_edges_common():

    # Realiza a ligação entre os patches da imagem
    source, target = get_connections_patches()

    # Realiza a ligação do vértice da imagem com todos os patches e vice-versa
    v_patches = list(range(1,5))
    v_img = [0]*4

    source = source + v_img
    target = target + v_patches
    source = source + v_patches
    target = target + v_img

    # Realiza a ligação do vértice da pergunta com os vértices da imagem e patches e vice-versa
    v_perg = [5]*5
    v_img_patches = list(range(0,5))

    source = source + v_perg
    target = target + v_img_patches
    source = source + v_img_patches
    target = target + v_perg

    # add self-loop desde o vértice da imagem até o vértice da pergunta
    source = source + list(range(0,6))
    target = target + list(range(0,6))

    return source, target

def define_edges(info_nlp, img, source_common, target_common):

    # Inicia a formação das arestas especificas de cada imagem

    source = source_common.copy()
    target = target_common.copy()

    # Realiza a conexão das palavras com a imagem, todos os patches da imagem, pergunta

    tam_perg = info_nlp[img]["len_perg"]
    node_img = list(range(6, 6+tam_perg))

    for n in node_img:

        source = source + [n]*6
        target = target + list(range(0,6))
        source = source + list(range(0,6))
        target = target + [n]*6

        # add self-loop in each word
        source = source + [n]
        target = target + [n]

    # Realiza a conexão das palavras com seus pares sintático/bidirecional

    for k in range(0, tam_perg):

        connections_words = info_nlp[img]["word_"+str(k)]["ligacoes"]
        connections_words = [i+6 for i in connections_words]

        for n2 in node_img:
            source = source + [n2]*len(connections_words)
            target = target + connections_words

    return source, target

def Get_graphs(type_base):

    data_list = []

    info_visao = loadData(type_base+"_info_visao.pkl")
    info_nlp = loadData(type_base+"_info_nlp")
    info_answers = loadData(type_base+"_info_answers")

    images = list(info_visao.keys())

    source_common, target_common = get_edges_common()

    for img in images:

        # features de visão + features de nlp
        features = np.concatenate((info_visao[img].detach().numpy(), info_nlp[img]["embeddings"]))

        # Calculate the L2 norm of the data
        #norm = np.linalg.norm(features)
        #normalized_data = features / norm

        #x = torch.tensor(normalized_data)
        x = torch.tensor(features)

        # Define as respostas referentes a cada grafo
        y = info_answers[img]
        y = pd.DataFrame(y).answer.values.tolist()

        # Cria as arestas
        source, target = define_edges(info_nlp, img, source_common, target_common)
        edge_index = torch.tensor([source, target], dtype=torch.long)

        # Cria os grafos
        data = Data(x=x, edge_index=edge_index, y=y)
        data_list.append(data)

    return data_list

In [ ]:
def save_graphs(dataset, name_arq):
    torch.save(dataset, name_arq+"_graphs.pt")
    return

In [ ]:
%%time

name_arq = "val"
dataset = Get_graphs(name_arq)
save_graphs(dataset, name_arq)

In [ ]:
idx = 5

In [ ]:
dataset[idx]

In [ ]:
dataset[idx].x

In [ ]:
dataset[idx].edge_index

In [ ]:
dataset[idx].y

In [ ]:
dataset[idx].num_features